In [1]:
SAT_FN = 'SAT_Results.csv'
HSD_FN = 'DOE_HIGH_SCHOOL_Directory_2014-2015.csv'
sc

In [2]:
import csv
with open (HSD_FN, 'rb') as fi:
    reader = csv.reader(fi)
    print reader.next()
    print reader.next()

['dbn', 'school_name', 'boro', 'building_code', 'phone_number', 'fax_number', 'grade_span_min', 'grade_span_max', 'expgrade_span_min', 'expgrade_span_max', 'bus', 'subway', 'primary_address_line_1', 'city', 'state_code', 'zip', 'website', 'total_students', 'campus_name', 'school_type', 'overview_paragraph', 'program_highlights', 'language_classes', 'advancedplacement_courses', 'online_ap_courses', 'online_language_courses', 'extracurricular_activities', 'psal_sports_boys', 'psal_sports_girls', 'psal_sports_coed', 'school_sports', 'partner_cbo', 'partner_hospital', 'partner_highered', 'partner_cultural', 'partner_nonprofit', 'partner_corporate', 'partner_financial', 'partner_other', 'addtl_info1', 'addtl_info2', 'start_time', 'end_time', 'se_services', 'ell_programs', 'school_accessibility_description', 'number_programs', 'priority01', 'priority02', 'priority03', 'priority04', 'priority05', 'priority06', 'priority07', 'priority08', 'priority09', 'priority10', 'Location 1']
['01M292', 'H

In [6]:
#UNIVOCITY - this library can take multiple lines, important library
df = spark.read \
    .format('com.databricks.spark.csv') \
    .option('header', 'true') \
    .option('inferSchema', 'true') \
    .option('parserLib', 'UNIVOCITY') \
    .load(HSD_FN)

NameError: name 'spark' is not defined

In [3]:
sat = sc.textFile(SAT_FN, use_unicode=False).cache()

In [4]:
list(enumerate(sat.first().split(',')))

[(0, 'DBN'),
 (1, 'SCHOOL NAME'),
 (2, 'Num of SAT Test Takers'),
 (3, 'SAT Critical Reading Avg. Score'),
 (4, 'SAT Math Avg. Score'),
 (5, 'SAT Writing Avg. Score')]

In [7]:
noHeaderSat = sat.filter(lambda x: not x.startswith('DBN,'))
noHeaderSat.first()

'02M047,47 THE AMERICAN SIGN LANGUAGE AND ENGLISH SECONDARY SCHOOL,16,395,400,387'

In [8]:
sat.getNumPartitions()

2

In [12]:
def extractScores(partitionId, partition):
    if partitionId==0:
        partition.next()
        
    import csv
    reader = csv.reader(partition)
    
    for row in reader:
        if row[2]!='s':
            (dbn, takers, score) = (row[0], int(row[2]), int(row[4]))
            yield (dbn, (score*takers, takers))

satScores = sat.mapPartitionsWithIndex(extractScores)
satScores.take(5)

[('02M047', (6400, 16)),
 ('21K410', (207575, 475)),
 ('30Q301', (43120, 98)),
 ('17K382', (22066, 59)),
 ('18K637', (13335, 35))]

In [13]:
schools = sc.textFile(HSD_FN, use_unicode=False).cache()
list(enumerate(schools.first().split('.')))

[(0,
  'dbn,school_name,boro,building_code,phone_number,fax_number,grade_span_min,grade_span_max,expgrade_span_min,expgrade_span_max,bus,subway,primary_address_line_1,city,state_code,zip,website,total_students,campus_name,school_type,overview_paragraph,program_highlights,language_classes,advancedplacement_courses,online_ap_courses,online_language_courses,extracurricular_activities,psal_sports_boys,psal_sports_girls,psal_sports_coed,school_sports,partner_cbo,partner_hospital,partner_highered,partner_cultural,partner_nonprofit,partner_corporate,partner_financial,partner_other,addtl_info1,addtl_info2,start_time,end_time,se_services,ell_programs,school_accessibility_description,number_programs,priority01,priority02,priority03,priority04,priority05,priority06,priority07,priority08,priority09,priority10,Location 1')]

In [15]:
def extractSchools(partitionId, partition):
    if partitionId==0:
        partition.next()
    import csv
    reader = csv.reader(partition)
    for row in reader:
        if len(row) == 58 and row[17].isdigit():
            (dbn, boro, total_students) = (row[0], row[2], int(row[17]))
            if total_students>500:
                yield (dbn, boro)
                
largeSchools = schools.mapPartitionsWithIndex(extractSchools)
largeSchools.take(5)

[('01M450', 'Manhattan'),
 ('01M539', 'Manhattan'),
 ('01M696', 'Manhattan'),
 ('02M374', 'Manhattan'),
 ('02M400', 'Manhattan')]

In [17]:
largeSchools.join(satScores).values().take(5) #values removes the left values

[('Bronx', (23069, 59)),
 ('Staten Island', (52216, 107)),
 ('Bronx', (16317, 49)),
 ('Brooklyn', (33235, 85)),
 ('Bronx', (24570, 65))]

In [19]:
largeSchools.join(satScores).values() \
    .reduceByKey(lambda x, y: (x[0]+y[0], x[1]+y[1])) \
    .mapValues(lambda x: x[0]/x[1]) \
    .take(5)

[('Bronx', 470),
 ('Manhattan', 514),
 ('Brooklyn', 487),
 ('Staten Island', 477),
 ('Queens', 474)]